In [1]:
from utils.dependency_setup import get_vector_store, get_embeddings
embeddings = get_embeddings()
vector_store = get_vector_store(embeddings, "langstuffindex")

In [21]:
from langchain.tools.retriever import create_retriever_tool
retriever = vector_store.as_retriever()
langgraph_glossary_retriever = create_retriever_tool(retriever=retriever, name="langgraph_glossary_retriever", description= "Contains all information about langgraph and langchain, use to clarify doubts or retrieve similar examples")

In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [40]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
("system", """You are a ReAct (Reasoning and Act) agent, expert at writing python code for implementing langgraph edges.
You are tasked with writing python code snippet that implements edges/conditional-edges given the description of the edge, the src and potential targets.

Information about how to implement edges/conditional-edges is below:
<Information>
***Edges***
Edges define how the logic is routed and how the graph decides to stop. This is a big part of how your agents work and how different nodes communicate with each other. There are a few key types of edges:

Normal Edges: Go directly from one node to the next.
Conditional Edges: Call a function to determine which node(s) to go to next.
A node can have MULTIPLE outgoing edges. If a node has multiple out-going edges, all of those destination nodes will be executed in parallel as a part of the next superstep.

***Normal Edges***
If you always want to go from node A to node B, you can use the add_edge method directly.

``` python
graph.add_edge("node_a", "node_b")
```

***Conditional Edges***
If you want to optionally route to 1 or more edges (or optionally terminate), you can use the add_conditional_edges method. This method accepts the name of a node and a "routing function" to call after that node is executed:

``` python
graph.add_conditional_edges("node_a", routing_function)
```

Similar to nodes, the routing_function accepts the current state of the graph and returns a value.

By default, the return value routing_function is used as the name of the node (or list of nodes) to send the state to next. All those nodes will be run in parallel as a part of the next superstep.

You can optionally provide a dictionary that maps the routing_function's output to the name of the next node.

``` python 
graph.add_conditional_edges("node_a", routing_function, {{True: "node_b", False: "node_c"}})
```
</Information>

Apart from langgraph edge/conditional_edge creation information, call the 'langgraph_glossary_retriever' tool to find for similar examples to the description of the input
Output: Code snippet, if conditional edge then also write the routing_function, the name of the function should be appropriate
"""),
("placeholder", "{messages}")])

In [ ]:
from langgraph.prebuilt import create_react_agent

edge_builder_agent = create_react_agent(model = llm, prompt= prompt, tools = [langgraph_glossary_retriever])